# UR SLS Sentinel 10m TC

Kode modelling klasifikasi densenet dan resnet urban rural pada data sentinel 2 10m dengan neighbor True crop, input tingkat SLS.

### Import Fast AI

In [1]:
import fastai
from fastai.vision.all import *
import pandas as pd

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score,balanced_accuracy_score
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/dataset/UR/05. Sentinel 10M SLS EN.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [5]:
import matplotlib.pyplot as plt
from sklearn import metrics

In [6]:
def plot_multroc(tst_preds, k =5):
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots( figsize=(9,7))
    for i in range(0,k):
        fpr, tpr, thresholds = metrics.roc_curve(tst_preds[i][1], tst_preds[i][0][:,1])
        roc_auc = metrics.auc(fpr, tpr)
        viz = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name="ROC fold {}".format(i))
        viz.plot(ax = ax)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.35,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="Receiver operating characteristic example",
    )
    ax.legend(loc="lower right")

    plt.show()

def cal_faccu(path , thres = 0.5):
  valt = []
  dataset = pd.read_csv(path)
  for i in range(1,6) :
    temp = dataset.loc[dataset.fold == i].copy()
    temp.loc["pred_lab"] = 0
    temp.loc[temp.pred > thres,"pred_lab"] = 1
    metri = [accuracy_score( temp.true,temp.pred_lab),balanced_accuracy_score( temp.true,temp.pred_lab),precision_score( temp.true,temp.pred_lab), recall_score( temp.true,temp.pred_lab),recall_score( temp.true,temp.pred_lab,pos_label = 0), f1_score( temp.true,temp.pred_lab),roc_auc_score( temp.true,temp.pred_lab)]
    valt.append(metri)
  print(pd.DataFrame(valt).mean())

### Preprocessing Image Input SLS

In [7]:
path = Path.cwd() / ("05. Sentinel 10M SLS EN")

### Densenet 

Pemodelan di tingat sls dengan densenet

In [8]:
avg = 'macro'
roca = RocAucBinary(average=avg)
f1s = F1Score(average=avg)
bc = BalancedAccuracy()
metricsc=[accuracy,bc, Precision(), Recall(),Recall(pos_label=0), f1s, roca]


In [ ]:
val_pct = []
tst_preds = []
datasetr = []
for kf in range(1,6):
  blocks = DataBlock(blocks = (ImageBlock, CategoryBlock),
                  get_items=get_image_files, # finds the images in the path
                  splitter=FileSplitter("/content/drive/MyDrive/Colab Notebooks/dataset/UR/kfold"+str(kf)+".txt"),  # default random split 80% training, 20% validation
                  get_y=parent_label # extracts the label category from the image's folder name 
                  ,item_tfms = Resize(64)
                  )
  dls = blocks.dataloaders(path,bs = 1000)
  learn = vision_learner(dls, densenet121, metrics=metricsc) 
  learn.fit_one_cycle(4,1e-2)
  preds,label = learn.get_preds(ds_idx  = 1)
  idx_test = []
  for i in dls.valid.items:
      idx_test.append(str(i)[-18:-4])
  true_label = []
  for i in dls.valid.items:
      if("urban" in str(i)) :
          true_label.append(1)
      else :
          true_label.append(0)

  test = pd.DataFrame(preds)
  test["idsls"] = idx_test
  test["true_lab"] = true_label
  test["iddesa"] = test.idsls.str[:10]
  test["stat"] = round(test[1])
  desa_pred_test = test.groupby('iddesa').agg({'stat': ['mean'], 'true_lab': 'mean'})
  desa_pred_test.columns = ["pred","true"]
  desa_pred_test["pred_lab"] = 0
  desa_pred_test["fold"] = kf
  datasetr.append(desa_pred_test)
  desa_pred_test.loc[desa_pred_test.pred > 0.825,"pred_lab"] = 1
  metri = [accuracy_score( desa_pred_test.true,desa_pred_test.pred_lab),balanced_accuracy_score( desa_pred_test.true,desa_pred_test.pred_lab),precision_score( desa_pred_test.true,desa_pred_test.pred_lab), recall_score( desa_pred_test.true,desa_pred_test.pred_lab),recall_score( desa_pred_test.true,desa_pred_test.pred_lab,pos_label = 0), f1_score( desa_pred_test.true,desa_pred_test.pred_lab),roc_auc_score( desa_pred_test.true,desa_pred_test.pred_lab)]
  val_pct.append(metri)
  tst_preds.append((TensorBase([1-desa_pred_test.pred,desa_pred_test.pred]).T , TensorCategory(desa_pred_test.true)))

In [ ]:
plot_multroc(tst_preds, k = 5)

In [ ]:
pd.DataFrame(val_pct).mean()

In [ ]:
dat = datasetr[0]
for i in range(1,5) :
  dat = pd.concat([dat,datasetr[i]],axis = 0,ignore_index = True)
dat.to_csv("/content/drive/MyDrive/Colab Notebooks/validate/05. fold densenet.csv")

In [ ]:
pd.DataFrame(val_pct).to_csv("/content/drive/MyDrive/Colab Notebooks/validate/05. densenet.csv")

In [ ]:
file_name = "/content/drive/MyDrive/Colab Notebooks/pickle predictions/05. densenet predictions.pkl"
with open(file_name, "wb") as fp:   #Pickling
   pickle.dump(tst_preds, fp)

In [ ]:
cal_faccu("/content/drive/MyDrive/Colab Notebooks/validate/05. fold densenet.csv",0.775)

### Resnet

In [ ]:
val_pct_rs = []
tst_preds_rs = []
datasetr_rs = []
for kf in range(1,6):
  blocks = DataBlock(blocks = (ImageBlock, CategoryBlock),
                  get_items=get_image_files, # finds the images in the path
                  splitter=FileSplitter("/content/drive/MyDrive/Colab Notebooks/dataset/UR/kfold"+str(kf)+".txt"),  # default random split 80% training, 20% validation
                  get_y=parent_label # extracts the label category from the image's folder name 
                  ,item_tfms = Resize(64)
                  )
  dls = blocks.dataloaders(path,bs = 1000)
  learn_rs = vision_learner(dls, resnet50, metrics=metricsc) 
  learn_rs.fit_one_cycle(4,1e-2)
  preds,label = learn_rs.get_preds(ds_idx  = 1)
  idx_test = []
  for i in dls.valid.items:
      idx_test.append(str(i)[-18:-4])
  true_label = []
  for i in dls.valid.items:
      if("urban" in str(i)) :
          true_label.append(1)
      else :
          true_label.append(0)

  test = pd.DataFrame(preds)
  test["idsls"] = idx_test
  test["true_lab"] = true_label
  test["iddesa"] = test.idsls.str[:10]
  test["stat"] = round(test[1])
  desa_pred_test = test.groupby('iddesa').agg({'stat': ['mean'], 'true_lab': 'mean'})
  desa_pred_test.columns = ["pred","true"]
  desa_pred_test["pred_lab"] = 0
  desa_pred_test["fold"] = kf
  datasetr_rs.append(desa_pred_test)
  desa_pred_test.loc[desa_pred_test.pred > 0.825,"pred_lab"] = 1
  metri = [accuracy_score( desa_pred_test.true,desa_pred_test.pred_lab),balanced_accuracy_score( desa_pred_test.true,desa_pred_test.pred_lab),precision_score( desa_pred_test.true,desa_pred_test.pred_lab), recall_score( desa_pred_test.true,desa_pred_test.pred_lab),recall_score( desa_pred_test.true,desa_pred_test.pred_lab,pos_label = 0), f1_score( desa_pred_test.true,desa_pred_test.pred_lab),roc_auc_score( desa_pred_test.true,desa_pred_test.pred_lab)]
  val_pct_rs.append(metri)
  tst_preds_rs.append((TensorBase([1-desa_pred_test.pred,desa_pred_test.pred]).T , TensorCategory(desa_pred_test.true)))

In [ ]:
plot_multroc(tst_preds_rs, k = 5)

In [ ]:
pd.DataFrame(val_pct_rs).mean()

In [ ]:
dat_rs = datasetr_rs[0]
for i in range(1,5) :
  dat_rs = pd.concat([dat_rs,datasetr_rs[i]],axis = 0,ignore_index = True)
dat_rs.to_csv("/content/drive/MyDrive/Colab Notebooks/validate/05. fold resnet.csv")

In [ ]:
pd.DataFrame(val_pct_rs).to_csv("/content/drive/MyDrive/Colab Notebooks/validate/05. resnet.csv")

In [ ]:
file_name = "/content/drive/MyDrive/Colab Notebooks/pickle predictions/05. resnet predictions.pkl"
with open(file_name, "wb") as fp:   #Pickling
   pickle.dump(tst_preds_rs, fp)

In [ ]:
cal_faccu("/content/drive/MyDrive/Colab Notebooks/validate/05. fold resnet.csv",0.775)